In [1]:
# Load libraries
if (!require("pacman")) install.packages("pacman")
library(pacman)
pacman::p_load("tidyverse", "psych", "data.table",  # Data wrangling and descriptive stats
               "epiDisplay", "sjstats")             # Reporting Odds Ratio and Std. Betas


# Load full dataset
df_clean <- read.csv("X:/BAR/Projects/SEISMIC/MeasurementGroup/AP project/Data/Derived/AP_processed/df_clean.csv")
head(df_clean)

Loading required package: pacman
Warning message:
"package 'pacman' was built under R version 3.6.3"Warning message:
"unable to access index for repository http://www.stats.ox.ac.uk/pub/RWin/bin/windows/contrib/3.6:
  cannot open URL 'http://www.stats.ox.ac.uk/pub/RWin/bin/windows/contrib/3.6/PACKAGES'"

package 'epiDisplay' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\feng8\AppData\Local\Temp\RtmpOAjDVs\downloaded_packages



epiDisplay installed
Warning message:
"package 'epiDisplay' was built under R version 3.6.3"also installing the dependencies 'rlang', 'estimability', 'mvtnorm', 'minqa', 'nloptr', 'statmod', 'RcppEigen', 'bayestestR', 'dplyr', 'effectsize', 'emmeans', 'insight', 'lme4', 'parameters', 'performance', 'sjlabelled', 'sjmisc'

Warning message:
"unable to access index for repository http://www.stats.ox.ac.uk/pub/RWin/bin/windows/contrib/3.6:
  cannot open URL 'http://www.stats.ox.ac.uk/pub/RWin/bin/windows/contrib/3.6/PACKAGES'"Warning message:
"package 'dplyr' is in use and will not be installed"

package 'rlang' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'rlang'"Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying D:\Anaconda3\envs\R-tutorial\Lib\R\library\00LOCK\rlang\libs\x64\rlang.dll to D:\Anaconda3\envs\R-tutorial\Lib\R\library\rlang\libs\x64\rlang.dll: Permission denied"Warning message:
"restored 'rlang'"

package 'estimability' successfully unpacked and MD5 sums checked
package 'mvtnorm' successfully unpacked and MD5 sums checked
package 'minqa' successfully unpacked and MD5 sums checked
package 'nloptr' successfully unpacked and MD5 sums checked
package 'statmod' successfully unpacked and MD5 sums checked
package 'RcppEigen' successfully unpacked and MD5 sums checked
package 'bayestestR' successfully unpacked and MD5 sums checked
package 'effectsize' successfully unpacked and MD5 sums checked
package 'emmeans' successfully unpacked and MD5 sums checked
package 'insight' successfully unpacked and MD5 sums checked
package 'lme4' successfully unpacked and MD5 sums checked
package 'parameters' successfully unpacked and MD5 sums checked
package 'performance' successfully unpacked and MD5 sums checked
package 'sjlabelled' successfully unpacked and MD5 sums checked
package 'sjmisc' successfully unpacked and MD5 sums checked
package 'sjstats' successfully unpacked and MD5 sums checked

The dow


sjstats installed
Warning message:
"package 'sjstats' was built under R version 3.6.3"Warning message in pacman::p_load("tidyverse", "psych", "data.table", "epiDisplay", :
"Failed to install/load:
sjstats"

st_id,firstgen,ethniccode,ethniccode_cat,female,lowincomflag,transfer,international,us_hs,cohort,...,summer_crs_1,enrl_from_cohort_1,numgrade_1,crs_term,summer_crs,enrl_from_cohort,numgrade,honor_1,skipped_course,discipline
2607686,0,Asian,2,1,0,0,0,1,2013,...,0,1.00,2.3,4152,0,1.33,0.7,0,0,BIO
2609268,1,White,0,0,0,0,0,1,2013,...,0,0.00,4.0,4152,0,1.33,3.3,0,0,BIO
2615928,1,White,0,1,1,0,0,1,2013,...,0,1.33,4.0,4158,0,2.00,4.0,0,0,BIO
2674044,0,White,0,1,1,0,0,1,2014,...,NA,NA,NA,4152,0,0.33,3.7,0,1,BIO
2743869,0,White,0,1,1,0,0,1,2014,...,0,1.00,3.7,4162,0,1.33,3.0,0,0,BIO
2744813,0,White,0,1,0,0,0,1,2014,...,0,0.33,2.3,4162,0,1.33,NA,0,0,BIO


In [2]:
# Subsets data for analysis 
# Took 2nd course in sequence
df_bio2 <- df_clean %>%
  subset(discipline == "BIO") 
# Took AP 
df_bio_aptakers <- df_clean %>%
  subset(discipline == "BIO") %>%
  subset(aptaker == 1)
# Skip eligible
df_bio_skeligible <- df_clean %>%
  subset(discipline == "BIO") %>%
  subset(eligible_to_skip == 1)

# Chem
# Took 2nd course in sequence
df_chem2 <- df_clean %>%
  subset(discipline == "CHEM") 
# Took AP 
df_chem_aptakers <- df_clean %>%
  subset(discipline == "CHEM") %>%
  subset(aptaker == 1)
# Skip eligible
df_chem_skeligible <- df_clean %>%
  subset(discipline == "CHEM") %>%
  subset(eligible_to_skip == 1)

# Phys
# Took 2nd course in sequence
df_phys2 <- df_clean %>%
  subset(discipline == "PHYS") 
# Took AP 
df_phys_aptakers <- df_clean %>%
  subset(discipline == "PHYS") %>%
  subset(aptaker == 1)
# Skip eligible
df_phys_skeligible <- df_clean %>%
  subset(discipline == "PHYS") %>%
  subset(eligible_to_skip == 1)

In [3]:
# Run Models (for each discipline)
# Note: For model specifications, check: https://docs.google.com/spreadsheets/d/1rN8W_iz1mr7lEzBGfdTZHa45wKOSLiSF8VEpChCPsmE/edit#gid=129222174
# BIO
# RQ1a - Who takes AP?
bio_rq1a <- glm(aptaker ~ 
                  factor(firstgen) + factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) +
                  scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(cohort), 
                data=df_bio2, binomial(link="logit"))
summary(bio_rq1a)
logistic.display(bio_rq1a)

# RQ1b - Who gets what score on AP exams? (DV: apscore, Sample: Took AP)
bio_rq1b <- lm(scale(apscore) ~ 
                 factor(firstgen) + factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) +
                 scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(cohort), 
               data=df_bio_aptakers)
summary(bio_rq1b)
effectsize::standardize_parameters(bio_rq1b)

# RQ1c.i - Of everyone, who earns an AP score that makes them eligible to skip? 
bio_rq1ci <- glm(eligible_to_skip ~ factor(firstgen) + factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) +
                   scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(cohort), 
                 data=df_bio2, binomial(link="logit"))
summary(bio_rq1ci)
logistic.display(bio_rq1ci)

# RQ1c.ii - Of those who take AP, who earns an AP score that makes them eligible to skip?
bio_rq1cii <- glm(eligible_to_skip ~ 
                    factor(firstgen) + factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) +
                    scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(cohort), 
                  data=df_bio_aptakers, binomial(link="logit"))
summary(bio_rq1cii)
logistic.display(bio_rq1cii)

# RQ2a - Of everyone who was eligible to skip, who actually skipped?
bio_rq2a <- glm(skipped_course ~ 
                  factor(firstgen) + factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) +
                  scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(crs_term), 
                data=df_bio_skeligible, binomial(link="logit"))
summary(bio_rq2a)
logistic.display(bio_rq2a)

# RQ2b - Of those who were eligible to skip, what was the effect of skipping on 2nd course grade (controlling for AP score)?
bio_rq2b <- lm(scale(numgrade) ~ factor(skipped_course) + scale(apscore) + 
                 factor(firstgen) + factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) + 
                 scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(crs_term), 
               data=df_bio_skeligible)
summary(bio_rq2b)

# RQ2c - Of everyone, what grade did they get in the 2nd course (controlling for AP score)? 
bio_rq2c <- lm(scale(numgrade) ~ factor(skipped_course) + scale(apscore) + 
                 factor(firstgen) + factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) + 
                 scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(crs_term), 
               data=df_bio2)
summary(bio_rq2c)


# RQ2d - Of everyone who took AP, what grade did they get in the 2nd course?
bio_rq2d <- lm(scale(numgrade) ~ factor(skipped_course) + 
                 factor(firstgen) + factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) + 
                 scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(crs_term), 
               data=df_bio_aptakers)
summary(bio_rq2d)


# RQ2e - Of everyone who took AP and skipped, what grade did they get in the 2nd course?
bio_rq2e <- lm(scale(numgrade) ~ factor(skipped_course) +
                 factor(firstgen) + factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) + 
                 scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(crs_term), 
               data=df_bio2)
summary(bio_rq2e)

  # use enrl_from_cohort as control
bio_rq2e_tmp <- lm(scale(numgrade) ~ factor(skipped_course) +
                 factor(firstgen) + factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) + 
                 scale(hsgpa) + scale(mathsr) + scale(englsr) +  factor(enrl_from_cohort), 
               data=df_bio2)
summary(bio_rq2e_tmp)

# RQ2f - Course grade, Skip eligible for those who took AP
bio_rq2f <- lm(scale(numgrade) ~ factor(eligible_to_skip) + 
                 factor(firstgen) + factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) + 
                 scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(crs_term), 
               data=df_bio_aptakers)
summary(bio_rq2f)


# RQ2g -  Skip eligible, full sample
bio_rq2g<- lm(scale(numgrade) ~ factor(eligible_to_skip) + 
                factor(firstgen) + factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) + 
                scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(crs_term), 
              data=df_bio2)
summary(bio_rq2g)


Call:
glm(formula = aptaker ~ factor(firstgen) + factor(lowincomflag) + 
    factor(gender) + factor(ethniccode_cat) + scale(hsgpa) + 
    scale(mathsr) + scale(englsr) + factor(cohort), family = binomial(link = "logit"), 
    data = df_bio2)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.9736  -0.9218  -0.6411   1.1070   2.4965  

Coefficients:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -1.12907    0.11945  -9.453  < 2e-16 ***
factor(firstgen)1       -0.08259    0.13331  -0.620 0.535568    
factor(lowincomflag)1   -0.03905    0.11164  -0.350 0.726478    
factor(gender)1         -0.04573    0.08988  -0.509 0.610915    
factor(ethniccode_cat)1  0.25928    0.13244   1.958 0.050269 .  
factor(ethniccode_cat)2  0.75780    0.12000   6.315 2.70e-10 ***
scale(hsgpa)             0.18063    0.05355   3.373 0.000743 ***
scale(mathsr)            0.24011    0.05944   4.040 5.35e-05 ***
scale(englsr)            0.47741    0.0597

,OR,lower95ci,upper95ci,Pr(>|Z|)
factor(firstgen)1,0.9207318,0.7090279,1.195647,5.355684e-01
factor(lowincomflag)1,0.9617011,0.7727091,1.196917,7.264783e-01
factor(gender)1,0.9553010,0.8010007,1.139325,6.109155e-01
factor(ethniccode_cat)1,1.2959907,0.9996956,1.680103,5.026928e-02
factor(ethniccode_cat)2,2.1335850,1.6864129,2.699330,2.703074e-10
scale(hsgpa),1.1979763,1.0786133,1.330548,7.431818e-04
scale(mathsr),1.2713942,1.1315867,1.428475,5.348633e-05
scale(englsr),1.6118927,1.4337328,1.812191,1.363080e-15
factor(cohort)2014,1.7427587,1.3307217,2.282376,5.437233e-05
factor(cohort)2015,1.6422990,1.2453927,2.165699,4.402263e-04



Call:
lm(formula = scale(apscore) ~ factor(firstgen) + factor(lowincomflag) + 
    factor(gender) + factor(ethniccode_cat) + scale(hsgpa) + 
    scale(mathsr) + scale(englsr) + factor(cohort), data = df_bio_aptakers)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.3104 -0.4788  0.0207  0.4892  2.4367 

Coefficients:
                        Estimate Std. Error t value Pr(>|t|)    
(Intercept)              0.28154    0.07351   3.830 0.000136 ***
factor(firstgen)1       -0.07038    0.08270  -0.851 0.394940    
factor(lowincomflag)1   -0.07399    0.06722  -1.101 0.271265    
factor(gender)1         -0.22009    0.05087  -4.326 1.66e-05 ***
factor(ethniccode_cat)1 -0.11338    0.08009  -1.416 0.157152    
factor(ethniccode_cat)2 -0.28101    0.06039  -4.653 3.69e-06 ***
scale(hsgpa)             0.08339    0.02845   2.932 0.003446 ** 
scale(mathsr)            0.24964    0.03426   7.288 6.20e-13 ***
scale(englsr)            0.38243    0.03274  11.681  < 2e-16 ***
factor(cohort)2014     

Parameter,Std_Coefficient
(Intercept),0.26164491
factor(firstgen)1,-0.06976071
factor(lowincomflag)1,-0.07333691
factor(gender)1,-0.21814760
factor(ethniccode_cat)1,-0.11238315
factor(ethniccode_cat)2,-0.27853447
scale(hsgpa),0.07828982
scale(mathsr),0.24744733
scale(englsr),0.37867243
factor(cohort)2014,-0.18019713



Call:
glm(formula = eligible_to_skip ~ factor(firstgen) + factor(lowincomflag) + 
    factor(gender) + factor(ethniccode_cat) + scale(hsgpa) + 
    scale(mathsr) + scale(englsr) + factor(cohort), family = binomial(link = "logit"), 
    data = df_bio2)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.9576  -0.5642  -0.3144  -0.1278   3.1540  

Coefficients:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -2.37899    0.17143 -13.877  < 2e-16 ***
factor(firstgen)1       -0.05262    0.20757  -0.253 0.799890    
factor(lowincomflag)1   -0.28808    0.16995  -1.695 0.090056 .  
factor(gender)1         -0.31111    0.11964  -2.600 0.009314 ** 
factor(ethniccode_cat)1  0.16146    0.20983   0.769 0.441598    
factor(ethniccode_cat)2  0.19301    0.14840   1.301 0.193394    
scale(hsgpa)             0.46772    0.10974   4.262 2.03e-05 ***
scale(mathsr)            0.59534    0.08271   7.198 6.12e-13 ***
scale(englsr)            0.93472 

...


Call:
lm(formula = scale(numgrade) ~ factor(skipped_course) + factor(firstgen) + 
    factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) + 
    scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(crs_term), 
    data = df_bio_aptakers)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.8957 -0.3776  0.1774  0.5594  2.8554 

Coefficients:
                        Estimate Std. Error t value Pr(>|t|)    
(Intercept)              0.14416    0.21938   0.657 0.511253    
factor(skipped_course)1 -0.14364    0.08544  -1.681 0.093034 .  
factor(firstgen)1       -0.05940    0.09542  -0.623 0.533722    
factor(lowincomflag)1   -0.25799    0.07685  -3.357 0.000817 ***
factor(gender)1         -0.02068    0.05811  -0.356 0.721987    
factor(ethniccode_cat)1 -0.05678    0.09201  -0.617 0.537329    
factor(ethniccode_cat)2  0.01668    0.06862   0.243 0.807959    
scale(hsgpa)             0.25205    0.03290   7.660 4.32e-14 ***
scale(mathsr)            0.24358    0.03923   6.209 7.7


Call:
lm(formula = scale(numgrade) ~ factor(skipped_course) + factor(firstgen) + 
    factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) + 
    scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(crs_term), 
    data = df_bio2)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.7315 -0.4234  0.1631  0.6014  2.6193 

Coefficients:
                        Estimate Std. Error t value Pr(>|t|)    
(Intercept)              0.21238    0.20775   1.022  0.30675    
factor(skipped_course)1 -0.08121    0.08284  -0.980  0.32701    
factor(firstgen)1       -0.12748    0.05355  -2.380  0.01736 *  
factor(lowincomflag)1   -0.11148    0.04512  -2.471  0.01354 *  
factor(gender)1         -0.06244    0.03684  -1.695  0.09024 .  
factor(ethniccode_cat)1 -0.05559    0.05322  -1.044  0.29637    
factor(ethniccode_cat)2  0.09541    0.04982   1.915  0.05557 .  
scale(hsgpa)             0.23454    0.02007  11.688  < 2e-16 ***
scale(mathsr)            0.17528    0.02471   7.092 1.69e-12 **


Call:
lm(formula = scale(numgrade) ~ factor(skipped_course) + factor(firstgen) + 
    factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) + 
    scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(enrl_from_cohort), 
    data = df_bio2)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.6927 -0.4297  0.2010  0.6170  2.7078 

Coefficients:
                             Estimate Std. Error t value Pr(>|t|)    
(Intercept)                   0.16287    0.53387   0.305  0.76034    
factor(skipped_course)1      -0.09605    0.08736  -1.100  0.27163    
factor(firstgen)1            -0.10032    0.05483  -1.830  0.06744 .  
factor(lowincomflag)1        -0.12776    0.04618  -2.767  0.00570 ** 
factor(gender)1              -0.05288    0.03772  -1.402  0.16104    
factor(ethniccode_cat)1      -0.05086    0.05447  -0.934  0.35049    
factor(ethniccode_cat)2       0.09319    0.05136   1.814  0.06974 .  
scale(hsgpa)                  0.24047    0.02068  11.631  < 2e-16 ***
scale(math


Call:
lm(formula = scale(numgrade) ~ factor(eligible_to_skip) + factor(firstgen) + 
    factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) + 
    scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(crs_term), 
    data = df_bio_aptakers)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.8126 -0.3938  0.1600  0.5633  2.7815 

Coefficients:
                           Estimate Std. Error t value Pr(>|t|)    
(Intercept)               -0.106928   0.221030  -0.484 0.628653    
factor(eligible_to_skip)1  0.264417   0.065832   4.017 6.34e-05 ***
factor(firstgen)1         -0.073823   0.094719  -0.779 0.435936    
factor(lowincomflag)1     -0.259241   0.076312  -3.397 0.000707 ***
factor(gender)1            0.004006   0.058066   0.069 0.945007    
factor(ethniccode_cat)1   -0.043969   0.091294  -0.482 0.630178    
factor(ethniccode_cat)2    0.055945   0.068330   0.819 0.413123    
scale(hsgpa)               0.243084   0.032782   7.415 2.56e-13 ***
scale(mathsr)             


Call:
lm(formula = scale(numgrade) ~ factor(eligible_to_skip) + factor(firstgen) + 
    factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) + 
    scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(crs_term), 
    data = df_bio2)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.6723 -0.4445  0.1676  0.6028  2.5734 

Coefficients:
                          Estimate Std. Error t value Pr(>|t|)    
(Intercept)                0.05567    0.20760   0.268  0.78859    
factor(eligible_to_skip)1  0.20661    0.04896   4.220 2.52e-05 ***
factor(firstgen)1         -0.12980    0.05337  -2.432  0.01508 *  
factor(lowincomflag)1     -0.10920    0.04498  -2.428  0.01525 *  
factor(gender)1           -0.05632    0.03675  -1.532  0.12557    
factor(ethniccode_cat)1   -0.06394    0.05309  -1.204  0.22857    
factor(ethniccode_cat)2    0.08219    0.04976   1.652  0.09875 .  
scale(hsgpa)               0.22943    0.02004  11.449  < 2e-16 ***
scale(mathsr)              0.15732    0.024

In [4]:

# cHEM
# RQ1a - Who takes AP?
chem_rq1a <- glm(aptaker ~ 
                  factor(firstgen) + factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) +
                  scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(cohort), 
                data=df_chem2, binomial(link="logit"))
summary(chem_rq1a)
logistic.display(chem_rq1a)

# RQ1b - Who gets what score on AP exams? (DV: apscore, Sample: Took AP)
chem_rq1b <- lm(scale(apscore) ~ 
                 factor(firstgen) + factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) +
                 scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(cohort), 
               data=df_chem_aptakers)
summary(chem_rq1b)
effectsize::standardize_parameters(chem_rq1b)

# RQ1c.i - Of everyone, who earns an AP score that makes them eligible to skip? 
chem_rq1ci <- glm(eligible_to_skip ~ factor(firstgen) + factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) +
                   scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(cohort), 
                 data=df_chem2, binomial(link="logit"))
summary(chem_rq1ci)
logistic.display(chem_rq1ci)

# RQ1c.ii - Of those who take AP, who earns an AP score that makes them eligible to skip?
chem_rq1cii <- glm(eligible_to_skip ~ 
                    factor(firstgen) + factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) +
                    scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(cohort), 
                  data=df_chem_aptakers, binomial(link="logit"))
summary(chem_rq1cii)
logistic.display(chem_rq1cii)

# RQ2a - Of everyone who was eligible to skip, who actually skipped?
chem_rq2a <- glm(skipped_course ~ 
                  factor(firstgen) + factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) +
                  scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(crs_term), 
                data=df_chem_skeligible, binomial(link="logit"))
table(df_chem_skeligible$skipped_course)
# NOT CONVERGE
# summary(chem_rq2a)
# logistic.display(chem_rq2a)

# RQ2b - Of those who were eligible to skip, what was the effect of skipping on 2nd course grade (controlling for AP score)?
chem_rq2b <- lm(scale(numgrade) ~ factor(skipped_course) + scale(apscore) + 
                 factor(firstgen) + factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) + 
                 scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(crs_term), 
               data=df_chem_skeligible)
# NOT CONVERGE
# summary(chem_rq2b)

# RQ2c - Of everyone, what grade did they get in the 2nd course (controlling for AP score)? 
chem_rq2c <- lm(scale(numgrade) ~ factor(skipped_course) + scale(apscore) + 
                 factor(firstgen) + factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) + 
                 scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(crs_term), 
               data=df_chem2)
summary(chem_rq2c)


# RQ2d - Of everyone who took AP, what grade did they get in the 2nd course?
chem_rq2d <- lm(scale(numgrade) ~ factor(skipped_course) + 
                 factor(firstgen) + factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) + 
                 scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(crs_term), 
               data=df_chem_aptakers)
summary(chem_rq2d)


# RQ2e - Of everyone who took AP and skipped, what grade did they get in the 2nd course?
chem_rq2e <- lm(scale(numgrade) ~ factor(skipped_course) +
                 factor(firstgen) + factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) + 
                 scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(crs_term), 
               data=df_chem2)
summary(chem_rq2e)

# RQ2f - Course grade, Skip eligible for those who took AP
chem_rq2f <- lm(scale(numgrade) ~ factor(eligible_to_skip) + 
                 factor(firstgen) + factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) + 
                 scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(crs_term), 
               data=df_chem_aptakers)
summary(chem_rq2f)


# RQ2g -  Skip eligible, full sample
chem_rq2g<- lm(scale(numgrade) ~ factor(eligible_to_skip) + 
                factor(firstgen) + factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) + 
                scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(crs_term), 
              data=df_chem2)
summary(chem_rq2g)


Call:
glm(formula = aptaker ~ factor(firstgen) + factor(lowincomflag) + 
    factor(gender) + factor(ethniccode_cat) + scale(hsgpa) + 
    scale(mathsr) + scale(englsr) + factor(cohort), family = binomial(link = "logit"), 
    data = df_chem2)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.5615  -0.6974  -0.4295  -0.0001   2.8387  

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -18.59172  287.48273  -0.065  0.94844    
factor(firstgen)1        -0.16122    0.16179  -0.996  0.31902    
factor(lowincomflag)1    -0.02012    0.13036  -0.154  0.87737    
factor(gender)1          -0.25407    0.10460  -2.429  0.01514 *  
factor(ethniccode_cat)1   0.24991    0.15593   1.603  0.10901    
factor(ethniccode_cat)2   0.36169    0.13566   2.666  0.00767 ** 
scale(hsgpa)              0.40878    0.07729   5.289 1.23e-07 ***
scale(mathsr)             0.41903    0.06897   6.076 1.24e-09 ***
scale(englsr)             0.3065

,OR,lower95ci,upper95ci,Pr(>|Z|)
factor(firstgen)1,8.511058e-01,6.198220e-01,1.168692e+00,3.190242e-01
factor(lowincomflag)1,9.800851e-01,7.591013e-01,1.265400e+00,8.773665e-01
factor(gender)1,7.756349e-01,6.318582e-01,9.521274e-01,1.514288e-02
factor(ethniccode_cat)1,1.283904e+00,9.458067e-01,1.742860e+00,1.090121e-01
factor(ethniccode_cat)2,1.435756e+00,1.100544e+00,1.873070e+00,7.671918e-03
scale(hsgpa),1.504974e+00,1.293415e+00,1.751137e+00,1.231779e-07
scale(mathsr),1.520490e+00,1.328239e+00,1.740567e+00,1.235321e-09
scale(englsr),1.358777e+00,1.186009e+00,1.556712e+00,9.932270e-06
factor(cohort)2014,1.306287e+07,2.571988e-238,6.634496e+251,9.545486e-01
factor(cohort)2015,3.479635e+07,6.851433e-238,1.767201e+252,9.518341e-01



Call:
lm(formula = scale(apscore) ~ factor(firstgen) + factor(lowincomflag) + 
    factor(gender) + factor(ethniccode_cat) + scale(hsgpa) + 
    scale(mathsr) + scale(englsr) + factor(cohort), data = df_chem_aptakers)

Residuals:
     Min       1Q   Median       3Q      Max 
-2.38974 -0.56904  0.01891  0.55690  2.38507 

Coefficients:
                        Estimate Std. Error t value Pr(>|t|)    
(Intercept)              0.19450    0.12112   1.606 0.108808    
factor(firstgen)1        0.06405    0.11802   0.543 0.587521    
factor(lowincomflag)1   -0.20272    0.09323  -2.175 0.030039 *  
factor(gender)1         -0.25377    0.06955  -3.649 0.000285 ***
factor(ethniccode_cat)1  0.07382    0.10891   0.678 0.498183    
factor(ethniccode_cat)2 -0.13359    0.08832  -1.512 0.130911    
scale(hsgpa)             0.06757    0.03709   1.822 0.068999 .  
scale(mathsr)            0.33514    0.04508   7.434 3.46e-13 ***
scale(englsr)            0.22249    0.04447   5.003 7.33e-07 ***
factor(cohor

Parameter,Std_Coefficient
(Intercept),0.22801972
factor(firstgen)1,0.06389460
factor(lowincomflag)1,-0.20222492
factor(gender)1,-0.25315455
factor(ethniccode_cat)1,0.07363538
factor(ethniccode_cat)2,-0.13326318
scale(hsgpa),0.06462002
scale(mathsr),0.33457386
scale(englsr),0.22194368
factor(cohort)2015,-0.13489642


Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"

...

,OR,lower95ci,upper95ci,Pr(>|Z|)
factor(firstgen)1,1.155426e-07,0.00000000,Inf,0.9948271565
factor(lowincomflag)1,6.759969e-08,0.00000000,Inf,0.9935034779
factor(gender)1,3.049050e-01,0.09986488,0.9309285,0.0370110178
factor(ethniccode_cat)1,3.041685e+00,0.56890594,16.2625292,0.1934171418
factor(ethniccode_cat)2,5.451776e-01,0.15006780,1.9805619,0.3566913796
scale(hsgpa),2.605058e+00,0.14029078,48.3733094,0.5206556413
scale(mathsr),3.812947e+00,1.40967735,10.3133985,0.0083817461
scale(englsr),5.787660e+00,2.07088408,16.1752216,0.0008132601
factor(cohort)2014,3.398097e+07,0.00000000,Inf,0.9951630517
factor(cohort)2015,2.555385e+07,0.00000000,Inf,0.9952425483


Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"


Call:
glm(formula = eligible_to_skip ~ factor(firstgen) + factor(lowincomflag) + 
    factor(gender) + factor(ethniccode_cat) + scale(hsgpa) + 
    scale(mathsr) + scale(englsr) + factor(cohort), family = binomial(link = "logit"), 
    data = df_chem_aptakers)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.0490  -0.1802  -0.0537  -0.0001   3.2772  

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)               -4.0850     0.9839  -4.152  3.3e-05 ***
factor(firstgen)1        -16.2202  2623.0122  -0.006  0.99507    
factor(lowincomflag)1    -15.8401  2203.8627  -0.007  0.99427    
factor(gender)1           -1.0812     0.6059  -1.784  0.07435 .  
factor(ethniccode_cat)1    0.7742     0.9247   0.837  0.40247    
factor(ethniccode_cat)2   -1.1688     0.6989  -1.672  0.09446 .  
scale(hsgpa)               0.4314     1.0631   0.406  0.68492    
scale(mathsr)              1.1108     0.4786   2.321  0.02028 *  
scale(englsr)  

,OR,lower95ci,upper95ci,Pr(>|Z|)
factor(firstgen)1,9.029412e-08,0.00000000,Inf,0.99506607
factor(lowincomflag)1,1.320539e-07,0.00000000,Inf,0.99426533
factor(gender)1,3.391766e-01,0.10343526,1.112201,0.07434893
factor(ethniccode_cat)1,2.168874e+00,0.35408141,13.285123,0.40246520
factor(ethniccode_cat)2,3.107281e-01,0.07896728,1.222683,0.09446228
scale(hsgpa),1.539357e+00,0.19161416,12.366628,0.68491769
scale(mathsr),3.036886e+00,1.18866471,7.758855,0.02028171
scale(englsr),5.081952e+00,1.83245795,14.093770,0.00178584
factor(cohort)2015,4.389586e-01,0.05543411,3.475923,0.43546121
factor(cohort)2016,1.591103e-01,0.01659387,1.525628,0.11099709


ERROR: Error in `contrasts<-`(`*tmp*`, value = contr.funs[1 + isOF[nn]]): contrasts can be applied only to factors with 2 or more levels


In [5]:

# PHYS
# RQ1a - Who takes AP?
phys_rq1a <- glm(aptaker ~ 
                   factor(firstgen) + factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) +
                   scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(cohort), 
                 data=df_phys2, binomial(link="logit"))
summary(phys_rq1a)
logistic.display(phys_rq1a)

# RQ1b - Who gets what score on AP exams? (DV: apscore, Sample: Took AP)
phys_rq1b <- lm(scale(apscore) ~ 
                  factor(firstgen) + factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) +
                  scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(cohort), 
                data=df_phys_aptakers)
summary(phys_rq1b)
effectsize::standardize_parameters(phys_rq1b)

# RQ1c.i - Of everyone, who earns an AP score that makes them eligible to skip? 
phys_rq1ci <- glm(eligible_to_skip ~ factor(firstgen) + factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) +
                    scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(cohort), 
                  data=df_phys2, binomial(link="logit"))
summary(phys_rq1ci)
logistic.display(phys_rq1ci)

# RQ1c.ii - Of those who take AP, who earns an AP score that makes them eligible to skip?
phys_rq1cii <- glm(eligible_to_skip ~ 
                     factor(firstgen) + factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) +
                     scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(cohort), 
                   data=df_phys_aptakers, binomial(link="logit"))
summary(phys_rq1cii)
logistic.display(phys_rq1cii)

# RQ2a - Of everyone who was eligible to skip, who actually skipped?
phys_rq2a <- glm(skipped_course ~ 
                   factor(firstgen) + factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) +
                   scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(crs_term), 
                 data=df_phys_skeligible, binomial(link="logit"))
table(df_phys_skeligible$skipped_course)
# NOT CONVERGE
summary(phys_rq2a)
logistic.display(phys_rq2a)

# RQ2b - Of those who were eligible to skip, what was the effect of skipping on 2nd course grade (controlling for AP score)?
phys_rq2b <- lm(scale(numgrade) ~ factor(skipped_course) + scale(apscore) + 
                  factor(firstgen) + factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) + 
                  scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(crs_term), 
                data=df_phys_skeligible)
# NOT CONVERGE
# summary(phys_rq2b)

# RQ2c - Of everyone, what grade did they get in the 2nd course (controlling for AP score)? 
phys_rq2c <- lm(scale(numgrade) ~ factor(skipped_course) + scale(apscore) + 
                  factor(firstgen) + factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) + 
                  scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(crs_term), 
                data=df_phys2)
summary(phys_rq2c)


# RQ2d - Of everyone who took AP, what grade did they get in the 2nd course?
phys_rq2d <- lm(scale(numgrade) ~ factor(skipped_course) + 
                  factor(firstgen) + factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) + 
                  scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(crs_term), 
                data=df_phys_aptakers)
summary(phys_rq2d)


# RQ2e - Of everyone who took AP and skipped, what grade did they get in the 2nd course?
phys_rq2e <- lm(scale(numgrade) ~ factor(skipped_course) +
                  factor(firstgen) + factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) + 
                  scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(crs_term), 
                data=df_phys2)
summary(phys_rq2e)

# RQ2f - Course grade, Skip eligible for those who took AP
phys_rq2f <- lm(scale(numgrade) ~ factor(eligible_to_skip) + 
                  factor(firstgen) + factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) + 
                  scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(crs_term), 
                data=df_phys_aptakers)
summary(phys_rq2f)


# RQ2g -  Skip eligible, full sample
phys_rq2g<- lm(scale(numgrade) ~ factor(eligible_to_skip) + 
                 factor(firstgen) + factor(lowincomflag) + factor(gender) + factor(ethniccode_cat) + 
                 scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(crs_term), 
               data=df_phys2)
summary(phys_rq2g)



Call:
glm(formula = aptaker ~ factor(firstgen) + factor(lowincomflag) + 
    factor(gender) + factor(ethniccode_cat) + scale(hsgpa) + 
    scale(mathsr) + scale(englsr) + factor(cohort), family = binomial(link = "logit"), 
    data = df_phys2)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.50473  -0.44905  -0.00011  -0.00003   2.66833  

Coefficients:
                          Estimate Std. Error z value Pr(>|z|)   
(Intercept)              -21.35103 1632.16850  -0.013  0.98956   
factor(firstgen)1          1.18843    0.60375   1.968  0.04902 * 
factor(lowincomflag)1     -0.13228    0.52105  -0.254  0.79959   
factor(gender)1            0.11113    0.39800   0.279  0.78009   
factor(ethniccode_cat)1   -1.56536    1.09528  -1.429  0.15295   
factor(ethniccode_cat)2    0.07438    0.46671   0.159  0.87337   
scale(hsgpa)               0.10839    0.25564   0.424  0.67156   
scale(mathsr)              0.69435    0.24490   2.835  0.00458 **
scale(englsr)         

,OR,lower95ci,upper95ci,Pr(>|Z|)
factor(firstgen)1,3.281923e+00,1.00512307,10.716118,0.049018835
factor(lowincomflag)1,8.760943e-01,0.31552632,2.432574,0.799590670
factor(gender)1,1.117535e+00,0.51224694,2.438051,0.780086463
factor(ethniccode_cat)1,2.090134e-01,0.02442691,1.788463,0.152952044
factor(ethniccode_cat)2,1.077221e+00,0.43155719,2.688877,0.873370269
scale(hsgpa),1.114487e+00,0.67526401,1.839400,0.671558407
scale(mathsr),2.002413e+00,1.23907714,3.236005,0.004578207
scale(englsr),1.991957e+00,1.21458019,3.266884,0.006331003
factor(cohort)2014,1.284086e+00,0.00000000,Inf,0.999916876
factor(cohort)2015,2.268119e+08,0.00000000,Inf,0.990594936



Call:
lm(formula = scale(apscore) ~ factor(firstgen) + factor(lowincomflag) + 
    factor(gender) + factor(ethniccode_cat) + scale(hsgpa) + 
    scale(mathsr) + scale(englsr) + factor(cohort), data = df_phys_aptakers)

Residuals:
    Min      1Q  Median      3Q     Max 
-1.8583 -0.5882  0.2070  0.4577  2.0665 

Coefficients:
                         Estimate Std. Error t value Pr(>|t|)   
(Intercept)              0.003218   0.264844   0.012  0.99036   
factor(firstgen)1       -0.720795   0.465647  -1.548  0.12914   
factor(lowincomflag)1    0.189988   0.478649   0.397  0.69343   
factor(gender)1          0.161514   0.287304   0.562  0.57699   
factor(ethniccode_cat)1 -1.020135   0.960952  -1.062  0.29449   
factor(ethniccode_cat)2 -0.081494   0.341651  -0.239  0.81263   
scale(hsgpa)            -0.125943   0.164708  -0.765  0.44876   
scale(mathsr)            0.513910   0.172295   2.983  0.00474 **
scale(englsr)            0.058991   0.174245   0.339  0.73663   
factor(cohort)2016    

Parameter,Std_Coefficient
(Intercept),0.08584710
factor(firstgen)1,-0.70140872
factor(lowincomflag)1,0.18487856
factor(gender)1,0.15717043
factor(ethniccode_cat)1,-0.99269780
factor(ethniccode_cat)2,-0.07930202
scale(hsgpa),-0.12118863
scale(mathsr),0.50452784
scale(englsr),0.05718319
factor(cohort)2016,0.12233205


Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"

...

Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"


 0  1 
 4 30 


Call:
glm(formula = skipped_course ~ factor(firstgen) + factor(lowincomflag) + 
    factor(gender) + factor(ethniccode_cat) + scale(hsgpa) + 
    scale(mathsr) + scale(englsr) + factor(crs_term), family = binomial(link = "logit"), 
    data = df_phys_skeligible)

Deviance Residuals: 
       Min          1Q      Median          3Q         Max  
-9.770e-06   2.110e-08   5.712e-07   3.587e-06   9.877e-06  

Coefficients:
                          Estimate Std. Error z value Pr(>|z|)
(Intercept)              1.740e+01  3.142e+05       0        1
factor(firstgen)1       -5.429e+02  9.725e+06       0        1
factor(lowincomflag)1    7.899e+02  1.496e+07       0        1
factor(gender)1          1.204e+00  2.215e+05       0        1
factor(ethniccode_cat)2 -4.829e+01  1.593e+05       0        1
scale(hsgpa)             1.529e+02  2.730e+06       0        1
scale(mathsr)            8.428e-01  1.826e+05       0        1
scale(englsr)           -1.287e+00  1.696e+05       0        1
factor(crs

,OR,lower95ci,upper95ci,Pr(>|Z|)
factor(firstgen)1,1.739805e-236,0,Inf,0.9999555
factor(lowincomflag)1,Inf,0,Inf,0.9999579
factor(gender)1,3.334017e+00,0,Inf,0.9999957
factor(ethniccode_cat)2,1.062562e-21,0,Inf,0.9997580
scale(hsgpa),2.653135e+66,0,Inf,0.9999553
scale(mathsr),2.322834e+00,0,Inf,0.9999963
scale(englsr),2.762090e-01,0,Inf,0.9999939
factor(crs_term)4162,2.241819e-35,0,Inf,0.9999068
factor(crs_term)4168,8.213260e+08,0,Inf,0.9999752
factor(crs_term)4172,3.588002e-13,0,Inf,0.9999652


ERROR: Error in `contrasts<-`(`*tmp*`, value = contr.funs[1 + isOF[nn]]): contrasts can be applied only to factors with 2 or more levels
